In [42]:
import pickle
import pandas as pd
import os
import setup

import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm.notebook import tqdm, trange
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import wandb
from torch.utils.data.sampler import SubsetRandomSampler
import pprint
import matplotlib.pyplot as plt
import torch

from sklearn.preprocessing import MinMaxScaler
import math
from torch.profiler import profile, record_function, ProfilerActivity

from operator import itemgetter
import operator
from random import randint
# from rnn_classes import Dog, DogInput, Race, Races, GRUNet, smallGRUNet, smalll_lin_GRUNet, smalll_prelin_GRUNet
import rnn_tools.rnn_classes as rnn_classes
from rnn_tools.raceDB import build_dataset
import importlib
import datetime
from rnn_tools.model_saver import model_saver, model_saver_wandb
import rnn_tools.training_testing_gru as training_testing_gru
from torch.nn.utils.rnn import pack_padded_sequence, pack_sequence, pad_packed_sequence,pad_sequence, unpack_sequence, unpad_sequence
import rnn_tools.training_testing_gru_double as training_testing_gru_double
import rnn_tools.training_testing_lstm as training_testing_lstm
from goto_conversion import goto_conversion

import rnn_tools.training_testing_gru_extra_data as training_testing_gru_extra_data
import rnn_tools.training_testing_stacking as training_testing_stacking
from torch.utils.data import Dataset, DataLoader

In [43]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")  # you can continue going on here, like cuda:1 cuda:2....etc.
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")

Running on the GPU


In [44]:
torch.set_default_tensor_type(torch.FloatTensor)

In [45]:
torch.__version__

'2.1.1'

In [46]:
['dist_last__1', 'box_last__1', 'speed_avg_1', 'split_speed_v1_1', 'split_speed_avg_1', 'split_margin_avg_1', 'margin_avg_1', 'margin_time_avg_1', 'RunHomeTime_1', 'run_home_speed_1', 'run_home_speed_v1_1', 'first_out_avg_1', 'pos_out_avg_1', 'post_change_avg_1', 'races_1', 'wins_1', 'wins_last_1', 'weight_', 'min_time_', 'min_split_time_', 'min_split_time_v1', 'last_start_price', 'last_start_prob']

['dist_last__1',
 'box_last__1',
 'speed_avg_1',
 'split_speed_v1_1',
 'split_speed_avg_1',
 'split_margin_avg_1',
 'margin_avg_1',
 'margin_time_avg_1',
 'RunHomeTime_1',
 'run_home_speed_1',
 'run_home_speed_v1_1',
 'first_out_avg_1',
 'pos_out_avg_1',
 'post_change_avg_1',
 'races_1',
 'wins_1',
 'wins_last_1',
 'weight_',
 'min_time_',
 'min_split_time_',
 'min_split_time_v1',
 'last_start_price',
 'last_start_prob']

In [47]:
def custom_l2(output, target):
    loss = torch.mean(abs(output-target), dim=1)
    return loss

In [48]:
def validation_CLE(x,y):
    loss_t = -torch.log(torch.exp(x)/torch.sum(torch.exp(x), dim=-1, keepdim=True))*y
    return loss_t

In [49]:
tracks = ['Sandown Park',
 'The Meadows',
 'Warragul',
 'Geelong',
 'Sale',
 'Ballarat',
 'Bendigo',
 'Cranbourne']

In [50]:
os.getcwd()
import rnn_tools.raceDB
importlib.reload(rnn_tools.raceDB)
# os.chdir(r"C:\Users\Nick\Documents\GitHub\grvmodel\Python\DATA")                                                                                                                                                                                                                                                                                                                             
date = datetime.datetime.strptime("2022-08-01", "%Y-%m-%d").date()
hidden_size = 32
states = ["NZ"]
data_file = './data/gru_inputs_kitchen_sink_new_not_simple.fth'
raceDB = rnn_tools.raceDB.build_dataset(data_file, hidden_size ,state_filter=states, margin_type='boosted_sftmin',v6=True)
raceDB.create_new_weights_v2()

c:\Users\Nick\Documents\GitHub\grvmodel\python


In [ ]:
raceDB.stats_cols = eval(raceDB.stats_cols)[0]

In [ ]:
# raceDB.stats_cols = ['box']+raceDB.stats_cols

In [ ]:
for i,s in enumerate(raceDB.stats_cols):
    print(i,s)

0 inside
1 midfield
2 wide
3 weight
4 DogGrade
5 _dist_last__1
6 _box_last__1
7 _speed_avg_1
8 _split_speed_avg_1
9 _split_margin_avg_1
10 _margin_avg_1
11 _margin_time_avg_1
12 _RunHomeTime_1
13 _run_home_speed_1
14 _first_out_avg_1
15 _pos_out_avg_1
16 _post_change_avg_1
17 _races_1
18 _wins_1
19 _wins_last_1
20 _weight_
21 _min_time_
22 _min_split_time_
23 _last_start_price
24 _last_start_prob
25 dist_dist_last__10
26 dist_box_last__10
27 dist_speed_avg_10
28 dist_split_speed_avg_10
29 dist_split_margin_avg_10
30 dist_margin_avg_10
31 dist_margin_time_avg_10
32 dist_RunHomeTime_10
33 dist_run_home_speed_10
34 dist_first_out_avg_10
35 dist_pos_out_avg_10
36 dist_post_change_avg_10
37 dist_races_10
38 dist_wins_10
39 dist_wins_last_10
40 dist_weight_
41 dist_min_time_
42 dist_min_split_time_
43 dist_last_start_price
44 dist_last_start_prob
45 box_dist_last__10
46 box_box_last__10
47 box_speed_avg_10
48 box_split_speed_avg_10
49 box_split_margin_avg_10
50 box_margin_avg_10
51 box_margi

In [ ]:
from rnn_tools.rnn_classes import Races,Dog, DogInput, Race

In [ ]:
import rnn_tools.raceDB

In [ ]:
stat_list = ['dist_last__1', 'box_last__1', 'speed_avg_1', 'split_speed_v1_1', 'split_speed_avg_1', 'split_margin_avg_1', 'margin_avg_1', 'margin_time_avg_1', 'RunHomeTime_1', 'run_home_speed_1', 'run_home_speed_v1_1', 'first_out_avg_1', 'pos_out_avg_1', 'post_change_avg_1', 'races_1', 'wins_1', 'wins_last_1', 'weight_', 'min_time_', 'min_split_time_', 'min_split_time_v1', 'last_start_price', 'last_start_prob']


In [ ]:
raceDB.create_test_split_date('2023-01-01',val_date='2023-07-01')
raceDB.create_dogs_test_split_date()

Train examples 15593, Test examples 2423, Val examples 2833


100%|██████████| 4028/4028 [00:00<00:00, 38581.08it/s]


In [ ]:
class BaggingDataset(Dataset):
    def __init__(self, race_ids, num_samples):
        self.data = race_ids
        self.num_samples = num_samples

    def __getitem__(self, index):
        # Sample a random index
        random_index = torch.randint(high=len(self.data), size=(1,)).item()
        return self.data[random_index]

    def __len__(self):
        return self.num_samples

In [ ]:
bagging_dataset = BaggingDataset(raceDB.train_race_ids, num_samples=1000)

In [ ]:
bagging_dataloader = DataLoader(bagging_dataset, batch_size=32)

In [ ]:
bagging_dataloader.__iter__().__next__()

['833066767',
 '768664847',
 '775513240',
 '716432184',
 '782154885',
 '620477980',
 '824847182',
 '570159712',
 '764579691',
 '509346729',
 '811481424',
 '665489991',
 '750654415',
 '504087612',
 '509334761',
 '579422602',
 '530406142',
 '707343118',
 '824820440',
 '550362205',
 '601881407',
 '775475296',
 '572604973',
 '605957520',
 '572605004',
 '546267409',
 '798946266',
 '827795858',
 '634917829',
 '504087178',
 '857139152',
 '580904662']

In [ ]:
def closure(optimizer, criterion, outs, classes):
    optimizer.zero_grad()
    loss = nn.functional.mse_loss(outs, classes)
    loss.backward()
    return loss

def model_pipeline(my_dataset=raceDB,config=None,prev_model=None, sweep=True, model_state_dict=None, prev_model_file=None, prev_model_version='450'):
    if my_dataset:
      dataset = my_dataset    
    else:
      dataset = raceDB
    # tell wandb to get started
    with wandb.init(project="NEW GRU V7_stacking", config=config,save_code=False):
      #  access all HPs through wandb.config, so logging matches execution!
      wandb.define_metric("loss_val", summary="min")
      wandb.define_metric("accuracy", summary="max")
      wandb.define_metric("ROI < 30", summary="max")
      wandb.define_metric("relu roi", summary="max")
      
      config = wandb.config
      pprint.pprint(config)
      pprint.pprint(config.epochs)
      print(config)
      wandb.run.log_code('rnn_tools/')
      # input_size = raceDB.get_race_input([0,1])[0].full_input.shape[0] #create fix so messy
      print(config.input_type)

      if 'batch_days' in config.keys():
        pass
        # raceDB.create_test_split_date(config['training_date_end'],val_date='2023-07-01')
        # raceDB.create_dogs_test_split_date()
        # raceDB.attach_races_to_dog_inputv2() 
        # raceDB.reset_hidden()


      for race in raceDB.racesDict.values():
        race.nn_input()

      input_size = raceDB.racesDict[raceDB.raceIDs[0]].full_input.shape
      print(f"input size: {input_size}")

      num_models = 20

      model = rnn_classes.AdvNet_stacking(input_size,num_models=num_models,fc0_size=config['f0_layer_size'], fc1_size=config['f1_layer_size'])
      bagging_dataset = BaggingDataset(raceDB.train_race_ids, num_samples=512)
      raceDB.dataloaders = []
      for i in range(num_models):
        raceDB.dataloaders.append(DataLoader(bagging_dataset, batch_size=128))

      optimizer = optim.AdamW(model.parameters(), lr=config['learning_rate'])
      raceDB.to_cuda()
      criterion = nn.CrossEntropyLoss(reduction='none')

      model = model.to(device)
      # print(model)
      
      try:
        training_testing_stacking.train_double_v3(model, dataset, criterion, optimizer, 'na', config)
      except (KeyboardInterrupt) as error:
        print(error)
        print("finished Early")
        
      # dataset.create_hidden_states_dict()
      raceDB.create_hidden_states_dict_v2()
      model_saver_wandb(model, optimizer, 450, 0.1, raceDB.hidden_states_dict_gru_v6,raceDB.train_hidden_dict , model_name="long nsw new  22000 RUN")
      if sweep:
        # raceDB.reset_all_lstm_states
        raceDB.reset_hidden()
    


    # and test its final performance
    #test(model, test_loader)

    return (model,dataset, optimizer)

In [ ]:
wandb_config_static = {
    'hidden_size': 256,
    'stats': raceDB.stats_cols,
    'races': states,
    'datafile': data_file,
    'latest_date': raceDB.latest_date,
    'input_type': 'basic',
    'num_layers': 2,
    'batch_size': 750,
    'dropout': 0.3,
    'epochs': 5000,
    'learning_rate': 0.0001,
    'optimizer': 'adamW',
    'f0_layer_size': 128,
    'f1_layer_size': 64,
    'training_date_end': '2023-01-01',
    'notes':'Testing for low loss NZ Model, only TD extra',
    'batch_days': 365,
    'dist_last__1': 1,  #
    'box_last__1': 1,  #
    'speed_avg_1': 1,  #
    'split_speed_v1_1': 0,  #
    'split_speed_avg_1': 0,  #
    'split_margin_avg_1': 1,
    'margin_avg_1': 1,  #
    'margin_time_avg_1': 1,  #
    'RunHomeTime_1': 1,
    'run_home_speed_1': 1,
    'run_home_speed_v1_1': 0,
    'first_out_avg_1': 1,  #
    'pos_out_avg_1': 1,  #
    'post_change_avg_1': 1,  #
    'races_1': 0,
    'wins_1': 0,
    'wins_last_1': 1,
    'weight_': 1,  #
    'min_time_': 1,  #
    'min_split_time_': 1,
    'min_split_time_v1': 0,  #
    'last_start_price': 0,  #
    'last_start_prob': 1,
}

In [ ]:
raceDB.reset_hidden(num_layers=wandb_config_static['num_layers'], hidden_size=wandb_config_static['hidden_size'])

In [ ]:
torch.cuda.empty_cache()

In [ ]:
raceDB.race_prices_to_prob()

In [ ]:
raceDB.latest_date

Timestamp('2024-01-31 00:00:00')

In [41]:
importlib.reload(training_testing_stacking)
importlib.reload(training_testing_lstm)
importlib.reload(training_testing_gru_double)
importlib.reload(rnn_tools.rnn_classes)
importlib.reload(rnn_classes)
importlib.reload(training_testing_gru_extra_data)
(model,dataset, optimizer) = model_pipeline(raceDB,config=wandb_config_static,sweep=False)
# all_price_df = model_pipeline(raceDB,config=wandb_config_static,sweep=False)


{'hidden_size': 256, 'stats': ['inside', 'midfield', 'wide', 'weight', 'DogGrade', '_dist_last__1', '_box_last__1', '_speed_avg_1', '_split_speed_avg_1', '_split_margin_avg_1', '_margin_avg_1', '_margin_time_avg_1', '_RunHomeTime_1', '_run_home_speed_1', '_first_out_avg_1', '_pos_out_avg_1', '_post_change_avg_1', '_races_1', '_wins_1', '_wins_last_1', '_weight_', '_min_time_', '_min_split_time_', '_last_start_price', '_last_start_prob', 'dist_dist_last__10', 'dist_box_last__10', 'dist_speed_avg_10', 'dist_split_speed_avg_10', 'dist_split_margin_avg_10', 'dist_margin_avg_10', 'dist_margin_time_avg_10', 'dist_RunHomeTime_10', 'dist_run_home_speed_10', 'dist_first_out_avg_10', 'dist_pos_out_avg_10', 'dist_post_change_avg_10', 'dist_races_10', 'dist_wins_10', 'dist_wins_last_10', 'dist_weight_', 'dist_min_time_', 'dist_min_split_time_', 'dist_last_start_price', 'dist_last_start_prob', 'box_dist_last__10', 'box_box_last__10', 'box_speed_avg_10', 'box_split_speed_avg_10', 'box_split_margin_a

  0%|          | 1/5000 [00:01<2:01:55,  1.46s/it]

output.shape=torch.Size([20, 2423, 8])
targets.shape=torch.Size([20, 2423, 8])
torch.Size([48460]) torch.Size([48460])
5952
stacked output.shape=torch.Size([48460, 8])
targets.shape=torch.Size([48460, 8])
6480


  0%|          | 4/5000 [00:05<1:49:51,  1.32s/it]

output.shape=torch.Size([20, 2423, 8])
targets.shape=torch.Size([20, 2423, 8])
torch.Size([48460]) torch.Size([48460])
6151
stacked output.shape=torch.Size([48460, 8])
targets.shape=torch.Size([48460, 8])
6480


...

100%|█████████▉| 4999/5000 [1:59:32<00:01,  1.38s/it]

output.shape=torch.Size([20, 2423, 8])
targets.shape=torch.Size([20, 2423, 8])
torch.Size([48460]) torch.Size([48460])
12272
stacked output.shape=torch.Size([48460, 8])
targets.shape=torch.Size([48460, 8])
12880


100%|██████████| 5000/5000 [1:59:34<00:00,  1.43s/it]


created path


loss,██▇▆▅▆▅▅▅▅▄▄▅▄▄▄▄▄▃▃▃▃▃▃▃▃▂▃▂▂▂▂▂▂▂▂▂▂▁▁
loss_stacked,█▇▇▇▇▆▆▅▅▄▄▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
stacked_accuracy,▁▇███▇▇▆▆▆▆▅▆▆▆▆▆▅▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆
test_accuracy,▁▃▅▆██████▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇
test_loss,█▆▃▂▁▁▁▁▂▂▂▂▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▄▃▄▄▄▄▄▄▄▄▄▄▄
val_loss_stacked,▂▁▁▁▁▁▂▂▃▃▄▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇███████
loss,1.74405
loss_stacked,0.44524
stacked_accuracy,0.26579
test_accuracy,0.25324
test_loss,1.98033


In [43]:
model

GRUNetv3_extra(
  (gru): GRU(26, 256, num_layers=2, dropout=0.3)
  (relu): ReLU()
  (fc0): Linear(in_features=2118, out_features=2048, bias=True)
  (batch_norm): BatchNorm1d(26, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batch_norm_data): BatchNorm1d(80, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (extra_1): GRUNetv3_simple_extra_data(
    (batch_norm): BatchNorm1d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
    (fc0_p1): Linear(in_features=20, out_features=128, bias=True)
    (fc0_p1_drop): Dropout(p=0.3, inplace=False)
    (fc0_p2): Linear(in_features=128, out_features=64, bias=True)
    (fc0_p2_drop): Dropout(p=0.3, inplace=False)
    (fc0_p3): Linear(in_features=64, out_features=64, bias=True)
    (fc0_p3_drop): Dropout(p=0.3, inplace=False)
  )
  (extra_2): GRUNetv3_simple_extra_data(
    (batch_norm): BatchNorm1d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): 

In [49]:
importlib.reload(training_testing_gru_extra_data)
training_testing_gru_extra_data.validate_model_v3(model,raceDB,config=wandb_config_static)

output.shape=torch.Size([2833, 8])
mean.shape=(2833, 8)
variance.shape=(2833, 8)
entropy.shape=(2833,)
mutual_info.shape=(2833,)
mean=array([[0.28174134, 0.21607923, 0.00346814, ..., 0.01175216, 0.00465347,
        0.30542733],
       [0.11105694, 0.30910803, 0.05616723, ..., 0.03895895, 0.04267633,
        0.05153325],
       [0.16246017, 0.06808614, 0.10166063, ..., 0.10107324, 0.03867751,
        0.20220393],
       ...,
       [0.18532827, 0.06536962, 0.12399554, ..., 0.10229177, 0.36441093,
        0.0264925 ],
       [0.03562071, 0.06281071, 0.20034123, ..., 0.21167989, 0.05063745,
        0.19286059],
       [0.20094422, 0.22126046, 0.26998586, ..., 0.02002967, 0.03805658,
        0.08065215]])
variance=array([[5.58260339e-03, 7.51602465e-03, 3.09334295e-05, ...,
        1.20065312e-04, 3.89783725e-05, 8.43325939e-03],
       [1.61427964e-03, 4.61224456e-03, 8.27125666e-04, ...,
        5.76935580e-04, 5.17855575e-04, 5.65828642e-04],
       [7.98621295e-04, 1.88691785e-04, 4.64

TypeError: 'NoneType' object is not callable

In [44]:
config = wandb_config_static

In [ ]:
model = rnn_classes.LSTMNetv3_BN_double(22,config['hidden_size'], num_layers=config['num_layers'],fc0_size=config['f0_layer_size'], fc1_size=config['f1_layer_size'])

In [ ]:
model = model.to('cuda:0')

In [ ]:
sweep_config = {"method": "grid"}

metric = {"name": "ROI < 30", "goal": "maximize"}

sweep_config["metric"] = metric


parameters_dict = {
    "optimizer": {"value": "adamW"},
    "batch_days": {"values": [90,180,365,550,10000]},
    "f0_layer_size": {"values": [128]},
    "f1_layer_size": {"values": [64]},
    "dropout": {"values": [0.3]},
    "input_type": {"values": ['basic']},
    "num_layers": {"values": [2]},
    'hidden_size':{'values':[128]},
    "len_data": {"value": len(raceDB.raceIDs)},
    "stats":{"value": raceDB.stats_cols},
    "races":{"value": states},
    "batch_size":{"value": 10},
}

sweep_config["parameters"] = parameters_dict

parameters_dict.update(
    {
        "epochs": {"values": [1000]},
        "validation_split": {"value": 0.1},
        "training_date_end": {"values": 
                                [
                                '2023-04-30',
                                '2023-02-28',
                                '2022-11-30',
                                '2021-11-30'
                                ]},
        "learning_rate": {"values": [0.0003, 0.0001]},
        "label_smoothing": {"values": [0.01,0.0]},
        "loss": {
            "values": [ "CEL"],
        },
    'dist_last__1': {'values': [0, 1]},
    'box_last__1': {'values': [0, 1]},
    'speed_avg_1': {'values': [0, 1]},
    'split_speed_v1_1': {'values': [0, 1]},
    'split_speed_avg_1': {'values': [0, 1]},
    'split_margin_avg_1': {'values': [0, 1]},
    'margin_avg_1': {'values': [0, 1]},
    'margin_time_avg_1': {'values': [0, 1]},
    'RunHomeTime_1': {'values': [0, 1]},
    'run_home_speed_1': {'values': [0, 1]},
    'run_home_speed_v1_1': {'values': [0, 1]},
    'first_out_avg_1': {'values': [0, 1]},
    'pos_out_avg_1': {'values': [0, 1]},
    'post_change_avg_1': {'values': [0, 1]},
    'races_1': {'values': [0, 1]},
    'wins_1': {'values': [0, 1]},
    'wins_last_1': {'values': [0, 1]},
    'weight_': {'values': [0, 1]},
    'min_time_': {'values': [0, 1]},
    'min_split_time_': {'values': [0, 1]},
    'min_split_time_v1': {'values': [0, 1]},
    'last_start_price': {'values': [0, 1]},
    'last_start_prob': {'values': [0, 1]}
}
)

import pprint

pprint.pprint(sweep_config)


sweep_config

In [ ]:
# wandb.agent('x9mz21wl', function=model_pipeline, count=100, project='NEW GRU-sweeps')

In [ ]:
sweep_id = wandb.sweep(sweep_config, project="Variable Sweeps")
# CUDA_LAUNCH_BLOCKING=1
wandb.agent(sweep_id, function=model_pipeline, count=1)